In [3]:
# 📦 1. Imports
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

# 🔌 2. Connexion à PostgreSQL
db_params = {
    "host": "localhost",
    "port": 5432,
    "dbname": "musesync",
    "user": "postgres",
    "password": "postgres"
}

engine = create_engine(
    f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
)

# ✅ Connexion test
pd.read_sql("SELECT version();", engine)


,version
0,"PostgreSQL 17.5 on x86_64-windows, compiled by..."


In [5]:
tables = ["artistes", "concerts", "ecoutes", "meteo"]

for table in tables:
    print(f"\n🧾 Aperçu de la table : {table}")
    display(pd.read_sql(f"SELECT * FROM {table} LIMIT 5", engine))



🧾 Aperçu de la table : artistes


,id,nom,genre,bpm_moyen,energie,popularite
0,1,Artiste_1,Jazz,131,0.72,88
1,2,Artiste_2,Folk,104,0.84,61
2,3,Artiste_3,Electro,114,0.86,25
3,4,Artiste_4,Folk,109,0.49,62
4,5,Artiste_5,Folk,116,0.56,57



🧾 Aperçu de la table : concerts


,id,id_artiste,lieu,date,genre,meteo,prix,nb_tickets_vendus
0,1,25,Le Rocher,2024-06-13,None,Ensoleille,10,118
1,2,22,Underground Club,2024-08-03,None,Canicule,18,132
2,3,27,Underground Club,2024-09-28,None,Ensoleille,20,60
3,4,18,L'Oasis,2024-07-21,None,Nuageux,15,227
4,5,16,L'Oasis,2024-08-12,None,Pluvieux,12,56



🧾 Aperçu de la table : ecoutes


,id,id_artiste,date,nombre_ecoutes,plateforme
0,1,21,2024-08-24,4809,Deezer
1,2,23,2024-07-26,6773,Apple Music
2,3,19,2024-08-09,12219,YouTube Music
3,4,3,2024-09-06,9300,Apple Music
4,5,7,2024-09-02,14111,Deezer



🧾 Aperçu de la table : meteo


,id,lieu,date,temperature,meteo
0,1,La Vapeur,2024-06-01,12,Nuageux
1,2,Le Rocher,2024-06-01,15,Ensoleille
2,3,L’Escale,2024-06-01,24,Canicule
3,4,Underground Club,2024-06-01,23,Brume
4,5,Nova Scene,2024-06-01,32,Pluvieux


In [7]:
# Vérif artistes inconnus dans ecoutes/concerts
query = """
SELECT e.id_artiste 
FROM ecoutes e
LEFT JOIN artistes a ON e.id_artiste = a.id
WHERE a.id IS NULL;
"""
df_invalid = pd.read_sql(query, engine)
print(f"❓ Artistes inconnus dans 'ecoutes' : {len(df_invalid)}")


❓ Artistes inconnus dans 'ecoutes' : 0


In [9]:
query = """
SELECT 
    a.nom AS artiste,
    DATE_TRUNC('month', e.date) AS mois,
    SUM(e.nombre_ecoutes) AS total_ecoutes
FROM ecoutes e
JOIN artistes a ON e.id_artiste = a.id
GROUP BY artiste, mois
ORDER BY mois, total_ecoutes DESC
"""
ecoutes_mensuelles = pd.read_sql(query, engine)
display(ecoutes_mensuelles.head())


,artiste,mois,total_ecoutes
0,Artiste_4,2024-05-31 22:00:00+00:00,460105
1,Artiste_23,2024-05-31 22:00:00+00:00,443039
2,Artiste_12,2024-05-31 22:00:00+00:00,413161
3,Artiste_18,2024-05-31 22:00:00+00:00,394696
4,Artiste_2,2024-05-31 22:00:00+00:00,374911


In [11]:
query = """
SELECT 
    lieu,
    ROUND(AVG(nb_tickets_vendus), 2) AS frequentation_moyenne
FROM concerts
GROUP BY lieu
ORDER BY frequentation_moyenne DESC
"""
frequentation_lieux = pd.read_sql(query, engine)
display(frequentation_lieux)


,lieu,frequentation_moyenne
0,L’Escale,278.53
1,Nova Scene,277.17
2,La Vapeur,270.40
3,Underground Club,253.64
4,L'Oasis,250.05
5,Le Rocher,238.72
6,Echo Hall,219.20


In [13]:
query = """
SELECT 
    c.meteo,
    COUNT(*) AS nb_concerts,
    ROUND(AVG(c.nb_tickets_vendus), 2) AS affluence_moyenne
FROM concerts c
GROUP BY c.meteo
ORDER BY affluence_moyenne DESC
"""
meteo_affluence = pd.read_sql(query, engine)
display(meteo_affluence)


,meteo,nb_concerts,affluence_moyenne
0,Orage,49,279.92
1,Pluvieux,58,268.40
2,Brume,51,264.53
3,Nuageux,42,242.98
4,Ensoleille,45,237.27
5,Canicule,55,230.75


In [15]:
query = """
SELECT 
    a.nom AS artiste,
    SUM(e.nombre_ecoutes) AS total_ecoutes,
    COUNT(DISTINCT c.id) AS nb_concerts,
    ROUND(AVG(c.nb_tickets_vendus), 2) AS affluence_moyenne
FROM artistes a
LEFT JOIN ecoutes e ON a.id = e.id_artiste
LEFT JOIN concerts c ON a.id = c.id_artiste
GROUP BY a.nom
ORDER BY total_ecoutes DESC
LIMIT 10
"""
top_artistes = pd.read_sql(query, engine)
display(top_artistes)


,artiste,total_ecoutes,nb_concerts,affluence_moyenne
0,Artiste_21,23359496,17,330.94
1,Artiste_12,21759780,15,235.87
2,Artiste_18,17763466,14,227.64
3,Artiste_11,17675745,15,181.87
4,Artiste_22,16517550,14,280.14
5,Artiste_30,16155156,12,159.92
6,Artiste_14,16095248,13,240.85
7,Artiste_2,15771679,11,314.18
8,Artiste_10,14971968,12,246.25
9,Artiste_6,14886982,11,255.64


In [19]:
ecoutes_mensuelles.to_csv("exports/ecoutes_mensuelles.csv", index=False)
frequentation_lieux.to_csv("exports/frequentation_lieux.csv", index=False)
meteo_affluence.to_csv("exports/meteo_affluence.csv", index=False)
top_artistes.to_csv("exports/top_artistes.csv", index=False)
print("📦 Données exportées dans le dossier 'exports'")


📦 Données exportées dans le dossier 'exports'
